In [9]:
#Packages being downloaded
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

from tkinter import filedialog
from tkinter import *

import os

In [70]:
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [69]:
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot()
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [68]:
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [67]:
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <101 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [66]:
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

In [65]:
def typeDefiner(df,catFound,sdtFile):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputCount=0
    for elem in root.findall('.//InputFields'):
        for inputs in elem.findall('.//InputField'):
            inputCount+=1
    
    if not catFound and inputCount>len(colNames):
        resetToDefault(sdtFile)
    print("input count is "+str(catFound)+str(len(colNames)))
    print("objects removed")
    print(colNames)
    return colNames

In [64]:
def toNumeric(sdtFile):

    #import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()
        

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
            
    
   #Try to write here
    for elem in root.findall('.//Glyph/Color/RGB/..'):
        check_req_elems = elem.find('.//Function')
        #check_req_elems.type = "Linear Interpolation"
        check_req_elems.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)
    print("color fixed")

In [80]:
def XMLParser(colNames, sdtFile,colorName,catFound,dict):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        if elem.get('name')!=dict['RGB']:
            elem.set('field', colNames[count])
            count+=1
        elif not catFound:
            elem.set('field', colNames[count])
            count+=1
        else:
            elem.set('field', colorName)
        
    #tree.write('New DAL 900.sdt')
    tree.write(sdtFile)
    print("XML updated")

In [63]:
def runAll(sdtFile):
    colorCol= None
    colorName=""
    catFound=False
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    print(colorName)
    if(colorName!=""):
        catFound=True
        colorCol=getCatCol(df,colorName)
    
    print(catFound)
    sdtFile=moveToNewFile(sdtFile,csv)
    colNames = typeDefiner(df,catFound,sdtFile)
    #colNames.append(colorName)
    dict =MakeDict('emptySDT.sdt')
    XMLParser(colNames, sdtFile,colorName,catFound,dict)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [88]:
#%%python
runAll('emptySDT.sdt')

reading csv done
Hitter_Team
True
input count is True13
objects removed
['id', 'dateTime', 'Inning', 'True_Distance', 'Speed_off_Bat', 'Elevation_Angle', 'Horizontal_Angle', 'Apex', 'Angle_Adj', 'Location_X', 'Location_Y', 'year', 'month']
['id', 'dateTime', 'Inning', 'True_Distance', 'Speed_off_Bat', 'Elevation_Angle', 'Horizontal_Angle', 'Apex', 'Angle_Adj', 'Location_X', 'Location_Y', 'year', 'month']
13
0
1
2
3
XML updated
Setting up Colors
98,133,8
170,18,187
40,118,229
205,47,247
122,25,202
95,156,217
23,51,15
208,76,163
183,115,150
6,15,173
91,38,14
6,246,214
179,95,148
112,60,74
240,60,132
176,240,204
177,23,157
170,40,121
227,86,76
197,148,201
198,48,139
221,155,69
151,119,232
81,153,156
20,24,149
214,155,93
1,138,110
6,1,95
62,146,30
140,48,224
212,58,178
46,30,6
233,70,5
201,232,253
115,78,215
147,246,30
140,117,30
94,131,214
14,55,212
137,35,28
133,30,250
162,193,52
18,35,0
230,169,226
47,17,82
141,136,227
61,209,35
102,211,37
108,98,112
218,118,62
219,167,38
117,148,147
18

# Extra functions

In [4]:
def elevationFinder(df):
    if df.columns.contains("elevation") or df.columns.contains("Elevation") or df.columns.contains("height"):
        longitude = df["Longitude"]
    else:
        longitude = "none found"
    return longitude

In [3]:
def latFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LAT', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass

In [2]:
def longFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LONG', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass

In [71]:
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#and then potentially add a binding field
        print("Changing to numeric")
        toNumeric(sdtFile)
        
    else:
        print("Setting up Colors")
        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in elem.findall('KeyValuePair/Key'):
                element.text=colorCol[x]
                x=x+1

            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                        pairs = elem.find('.//KeyValuePairs')
                #fixes spacing so new pair will be at right column/indent area
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail="\n\t\t\t\t\t\t\t"
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text="\n\t\t\t\t\t\t\t\t"
                        keyVal.tail="\n\t\t\t\t\t\t"
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail="\n\t\t\t\t\t\t\t\t"
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail="\n\t\t\t\t\t\t\t"
                        x=x+1
    
                        x=0

                        count=0
                        colorList=[] 
                        while count<len(colorCol):
                            randomCol(colorList)
                            count=count+1


                        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
                            for element in elem.findall('KeyValuePair/Value'):
                                element.text=colorList[x]
                                x=x+1
                
                
            file.write(sdtFile,encoding="utf-8", xml_declaration=True)


In [72]:
#
def moveToNewFile(sdtFile,csv):
    file = ET.parse(sdtFile)
    base=os.path.basename(csv)
    newFile =os.path.splitext(base)[0]+".sdt"
    file.write(newFile,encoding="utf-8", xml_declaration=True)
    
    return newFile

In [73]:
def randomCol(colorList):
    import random
    r = random.random()*255
    r=int(r)
    g = random.random()*255
    g=int(g)
    b = random.random()*255
    b=int(b)
    color=str(r)+","+str(g)+","+str(b)
    colorList.append(color)

In [74]:
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    print("resetting")
    element=None
    for parent in root.findall('.//Glyph/Color/RGB'):
        for e in parent:
            print(e)
            if(element==None):
                element = parent.find('Binding')
                print(element)
        if(element!=None):
            parent.remove(element)
            element=None
    
    for parent in root.findall('.//InputFields'):
        for e in parent:
            print(e)
            if(element==None):
                element =  parent.find('InputField[@name="color"]')
                print(element)
                print("Element found")
        if (element!=None):
            parent.remove(element)
            element=None
    
    count=0
    for parent in root.findall('.//InputFields'):
        for e in parent:
            if(count==len(parent)-1):
                e.tail="\n\t"
            count+=1
        
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [59]:
def MakeDict(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputs={'Name':'Id'}
    for parent in root.findall('.//Glyph'):
        for elem in parent:
            for e in elem:
                for l in e:
                    temp=e.find('Binding')
                    if temp!=None:
                        inputs[e.tag]=temp.get('fieldId')
                        temp=None
                        
    return inputs

In [89]:
dict =MakeDict('emptySDT.sdt')
print(dict)

{'Name': 'Id', 'X': '101', 'Y': '111', 'Z': 'z', 'RGB': '555'}
